NOMBRE: ALEX HELDER HUANCARA CCOLQQUE

Importamos las librerías a usar 

In [32]:
import pandas as pd
import numpy as np

Usaremos esta funcion para crear combinaciones de una entrada que nos devuelve subsecuencias de longitud r, si los elementos de la entrada son únicos, no habrá valores repetidos en las combinaciones.

In [33]:
def combinaciones(entrada, long_salida):
    aux = tuple(entrada)
    n = len(aux)
    if long_salida > n:
        return
    indices = list(range(long_salida))
    yield tuple(aux[i] for i in indices)

    while True:
        for i in reversed(range(long_salida)):
            if indices[i] != i + n - long_salida:
                break
        else:
            return
        indices[i] += 1
        for j in range(i + 1, long_salida):
            indices[j] = indices[j - 1] + 1
        yield tuple(aux[i] for i in indices)

Prueba de las combinaciones

In [17]:
tupaaa = combinaciones('abcde', 3)
for x in tupaaa:
  print(x)
  

('a', 'b', 'c')
('a', 'b', 'd')
('a', 'b', 'e')
('a', 'c', 'd')
('a', 'c', 'e')
('a', 'd', 'e')
('b', 'c', 'd')
('b', 'c', 'e')
('b', 'd', 'e')
('c', 'd', 'e')


# Implementacion del algoritmo Apriori

In [119]:
class Apriori:
  # inilcializacion de atributos
  def __init__(self, datos,  min_support = 0, confidence = 0, lift = 0):
    self.datos = datos
    self.min_support = min_support
    self.confidence = confidence
    self.lift = lift

  """ Esta funcion nos permite tranformar el dataset.npy en una lista, además
  elimina las canciones repetidas (transformacion en conjunto) y ademas
  almacenamos en una variable las veces que una cancion se repite dentro de la playlist"""
  def preparar_datos(self):
    self.playlists = list(self.datos.item().values()) # convertir en lista
    self.playlists = [set(playlist) for playlist in self.playlists] # convertimos en conjunto para eliminar musicas repetidas
    canciones = [item for sublista in self.playlists for item in sublista]
    self.songs_counter = pd.Series(data=canciones).value_counts().to_dict()

  """Con este método generamos un diccionario con las canciones y los indices 
  de las listas"""
  import collections
  def obtener_apariciones_canciones(self):
    canciones_en_playlists = collections.defaultdict(set) # Creamos un dicionario de tuplas
    for indice, playlist in enumerate(self.playlists): #recorremos las playlists
      for cancion in playlist: #Para cada cancion agregamos sus indices en el diccionario
          canciones_en_playlists[cancion].add(indice)
    self.songs_in_playlists = canciones_en_playlists

  """ Esta funcion nos genera itemset frecuentes largos, aquellos itemsets que 
  que aparecen mas veces que el min_support"""
  def generate_L_1(self):
    #verificamos que se agreguen canciones que sobrepasen el min_support
    self.L_1_counter = { cancion: tiempos for cancion, tiempos in self.songs_counter.items() if tiempos / len(self.playlists) >= self.min_support }
    self.L_1 = [{cancion} for cancion in self.L_1_counter.keys()]


  """Esta funcion nos generará posibles itemsets frecuentes, para eso usaremos 
  la union de conjuntos"""
  def posibles_itemsets_frecuentes(self, current_itemsets, k):
    C_k = set()
    m = k - 2
    for candidato in current_itemsets:
      candidato = list(candidato) # convertimos en lista los posibles items sets frecuentes
      for aux_candidate in current_itemsets:
        aux_candidate = list(aux_candidate)
        can_join = True
        for i in range(k - 2):
          if candidato[i] != aux_candidate[i]:
            can_join = False
            break
        if not can_join:
          continue
        if candidato[k - 2] < aux_candidate[k - 2]:
          c = candidato + [aux_candidate[k - 2]]
          c = frozenset(sorted(c))
          C_k.add(c)
    return C_k
  
  """ obtenemos el min_support de itemset e intersectamos con las playlists """
  def calculate_subset_count(self, subset):
    playlists_inter = []
    for song in subset:
      playlists_inter.append(self.songs_in_playlists[song])
    return len(set.intersection(*playlists_inter))

  """ Nos retorna un conjuntos de itemsets que hayan superado el min_support"""
  def podar_itemsets(self, C_k):
    C_k_counter = {}
    playlist_length = len(self.playlists)
    for candidate in C_k:
      C_k_counter[candidate] = self.calculate_subset_count(candidate)
    L_k_counter = { subset: times for subset, times in C_k_counter.items() if times / playlist_length >= self.min_support}
    return L_k_counter

  """ Esta funcion nos permite exportar los itemsets frecuentes a csv, para
  poder verlo con facilidad y mas legible"""
  def export_frequent_itemsets_to_csv(self):
    self.frequent_itemsets_df = pd.DataFrame([item for sublist in self.k_frequent_itemsets.values() for item in sublist]).round(3)
    self.frequent_itemsets_df.columns = ["itemset", "count_support"]
    self.frequent_itemsets_df["support"] = self.frequent_itemsets_df["count_support"] / len(self.playlists) # support
    print("Generando archivo de itemsets frecuentes.") # mostramos mensaje de informacion
    self.frequent_itemsets_df.to_csv("itemsets_frecuentes.csv", index="False") # guardamos los itemsets frecuentes en el archivo.csv
    return self.frequent_itemsets_df

  """ Este método nos genera los itemsets frecuentes"""
  def get_frequent_itemsets(self):
    self.preparar_datos() #Preparamos los datos de la playlists
    self.obtener_apariciones_canciones() # obtenemos itemsets frecuentes
    self.generate_L_1() # generamos las itemsets que superen el min_support
    self.k_frequent_itemsets = {}
    self.k_frequent_itemsets[1] = sorted(self.L_1_counter.items(), key=lambda x: x[1], reverse=True)
    self.frequent_itemsets_length_gt2 = []
    k = 2
    current = self.L_1
    while len(current) != 0:
      C_k = self.posibles_itemsets_frecuentes(current, k)
      L_k_counter = self.podar_itemsets(C_k)
      L_k = L_k_counter.keys()
      self.frequent_itemsets_length_gt2.extend(L_k)
      self.k_frequent_itemsets[k] = sorted(
        L_k_counter.items(), key=lambda x: x[1], reverse=True)
      k += 1
      current = L_k
    self.export_frequent_itemsets_to_csv()

  """Nos forma todas las combianciones de un itemset"""
  def get_association_rule_from_itemset(self, itemset):
    itemset_count = self.calculate_subset_count(itemset)
    itemset_support = itemset_count / len(self.playlists)
    for i in range(1, len(itemset) + 1):
      for x_set in combinaciones(itemset, i):
        x_set = set(x_set)
        y_set = set(itemset) - x_set
        x_support = self.calculate_subset_count(x_set) / len(
          self.playlists)
        x_y_support = self.calculate_subset_count(
          x_set.union(y_set)) / len(self.playlists)
        rule_confidence = x_y_support / x_support
        if len(x_set) > 0 and len(y_set) > 0:
          y_support = self.calculate_subset_count(y_set) / len(
            self.playlists)
          rule_lift = x_y_support / (x_support * y_support)
          self.rules.append((x_set, y_set, rule_confidence, x_y_support, rule_lift))

  """ Este modulo genera todas las reglas de asociaciion posibles, todo esto a partir 
  de los itemsets frecuentes"""
  def generate_association_rules(self):
    self.rules = [] # inicializamos un arreglo vacio
    for itemset in self.frequent_itemsets_length_gt2: 
      self.get_association_rule_from_itemset(itemset) # obtenemos las reglas de asociacion para cada itemset
    self.exportar_reglas() # Exportamos las reglas de asociacion

  """ Este modulo nos sirve pare exportar las reglas de asociacion al formato csv utilizando pandas"""
  def exportar_reglas(self):
    self.rules_df = pd.DataFrame(data=self.rules, columns=["anterior", "siguiente", "confidence", "support", "lift"]).round(3) # creamos un data frame con las etiquetas
    print("Generando archivo de reglas de asociacion.") # Mensaje de informacion
    self.rules_df["anterior"] = list(map(tuple, self.rules_df["anterior"])) # mapeamos los anteriores
    self.rules_df["siguiente"] = list(map(tuple, self.rules_df["siguiente"])) #mapeamos los siguientes
    self.rules_df.to_csv("reglas_de_asociacion.csv", index=False) # guardamos el data frame en un archivo .csv
    return self.rules_df

  """ Este modulo nos reportar los n reglas de asociacion con mayor confidencia, pero podemos cambiar a lift"""
  def obtener_reglas_asociaciacion(self, n, ordenado_por="confidence"):
    sorted_data_frame = self.rules_df.sort_values(ordenado_por, ascending=False).head(n) # data frame
    print("Genrando top {} reglas ordenado por {}.".format(n, ordenado_por)) # mostrar mensaje de informacion
    sorted_data_frame.to_csv("top_{}_rule_by_{}.csv".format(n, ordenado_por), index=False) # generar un archivo .csv del data frame ordenado
    return sorted_data_frame

  """Este modulo nos permite filtrar todas las reglas segun condifence y lift o ambos por separado"""
  def filtrar_reglas(self, filtrar_por="ambos"):
    if filtrar_por == "ambos": # ambos significa confidence y lift
      filtered_rules = self.rules_df[(self.rules_df["confidence"] >= self.confidence)& (self.rules_df["lift"] >= self.lift)]
    elif filtrar_por == "confidence": # filtrar solo por confidence
      filtered_rules = self.rules_df[self.rules_df["confidence"] >= self.confidence].sort_values(filtrar_por, ascending=False)
    else:
      filtered_rules = self.rules_df[
        self.rules_df["lift"] >= self.lift].sort_values(filtrar_por, ascending=False)
    display(filtered_rules.head(10)) # filtrar solo 10 reglas
    filtered_rules.to_csv("reglas_filtradas.csv", index=False)
    print("Primeras 1o reglas de asociacion filtradas")

Cargar los datos de las playlists

In [120]:
spotify = np.load("spotify.npy", allow_pickle=True)

Inicializar la clase Apriori para poder usar sus métodos

In [121]:
apriori = Apriori( datos=spotify, min_support=0.01, confidence=0.5, lift=1.2)

Primero obtendremos los itemsets frecuentes para poder obtener las reglas de asociacion

In [122]:
apriori.get_frequent_itemsets()

Generando archivo de itemsets frecuentes.


Ahora procedemos a generar las reglas de asociacion.

In [123]:
apriori.generate_association_rules()

Generando archivo de reglas de asociacion.


Finalmente mostramos las 10 primeras de asociacion, para ello tenemos que pasar de parametro n = 10, y las reglas de asociacion estan ordenadas según la confianza para saber la probabilidad que que la musica siguiente se escuche, una vez terminado.

In [124]:
apriori.obtener_reglas_asociaciacion(10)


Genrando top 10 reglas ordenado por confidence.


,anterior,siguiente,confidence,support,lift
446,"(Mask Off, DNA.)","(HUMBLE.,)",0.909,0.010,19.550
410,"(XO TOUR Llif3, DNA.)","(HUMBLE.,)",0.864,0.010,18.589
21,"(DNA.,)","(HUMBLE.,)",0.823,0.019,17.688
367,"(Mask Off, XO TOUR Llif3)","(HUMBLE.,)",0.804,0.013,17.283
385,"(Broccoli (feat. Lil Yachty), Bounce Back)","(Bad and Boujee (feat. Lil Uzi Vert),)",0.775,0.010,22.469
415,"(XO TOUR Llif3, Slippery (feat. Gucci Mane))","(HUMBLE.,)",0.765,0.010,16.455
392,"(XO TOUR Llif3, Tunnel Vision)","(HUMBLE.,)",0.750,0.010,16.129
379,"(Mask Off, Congratulations)","(HUMBLE.,)",0.747,0.012,16.063
421,"(Mask Off, Bounce Back)","(HUMBLE.,)",0.743,0.010,15.971
397,"(Mask Off, goosebumps)","(HUMBLE.,)",0.743,0.011,15.984


Escogeremis las 4 primeras regla.
El soporte nos indica la frecuencia en que la musica aparece en las playlists.La confianza nos indica la probabilidad de que escuchemos la cancion siguiente.
Y el lift la probabilidad de que escuchemos una cancion siguiente, uan vez escuchada la cancion actual dependiendo de la popularidad de la música.

In [125]:
apriori.obtener_reglas_asociaciacion(4)

Genrando top 4 reglas ordenado por confidence.


,anterior,siguiente,confidence,support,lift
446,"(Mask Off, DNA.)","(HUMBLE.,)",0.909,0.010,19.550
410,"(XO TOUR Llif3, DNA.)","(HUMBLE.,)",0.864,0.010,18.589
21,"(DNA.,)","(HUMBLE.,)",0.823,0.019,17.688
367,"(Mask Off, XO TOUR Llif3)","(HUMBLE.,)",0.804,0.013,17.283


In [ ]:
"""def generate_association_rules(frequent_itemsets, confidence = 0, lift=0):
  support = []
  m = len(frequent_itemsets)
  for item in frequent_itemsets[1]:
    support_aux = []
    n = len(item)
    for i in range(item):
      support_aux.append(item[i] / frequent_itemsets[2])
    support.append(support_aux)

  items_frequents = frequent_itemsets[0][m-1]
  n_itemsFreq = len(items_frequents)
  for i in range(n_itemsFreq):
    print("Item: ", items_frequents[1])
    rules_itemsets = generate_rules(items_frequents[i])
    rules_lift = []
    rules_confidence = [] """